In [ ]:
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import itertools
import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
# Apparently, SNS stands for "Samuel Norman Seaborn", a fictional
# character from The West Wing
import seaborn as sns
import sympy

sns.set()
sympy.init_printing()
# Make the figures directory if it doesn't exist.
Path('figures/').mkdir(exist_ok=True)

In [ ]:
def generate_individual(n, scale=100):
    """Generates an individual solution of the given size.

    Pick random coordinates in the `scale`x`scale` grid uniformly.

    :param n: The size of the individual to generate.
    :param scale: How much to scale the individual's coordinates by.
    """
    # Scale up the uniform values from [0, 1].
    return np.random.rand(n, 2) * scale

def generate_population(size, n, scale=100):
    """Generate a population of individuals with the given size.

    Pick random coordinates in the `scale`x`scale` grid uniformly.

    :param size: The number of individuals to generate.
    :param n: The size of each individual.
    :param scale: How much to scale the individual's coordinates by.
    """
    # Scale up the uniform values from [0, 1].
    return np.random.rand(size, n, 2) * scale

In [ ]:
PROBLEM_SIZE = 50
individual = generate_individual(PROBLEM_SIZE)
plt.plot(individual[:, 0], individual[:, 1], 'r')
plt.plot(individual[:, 0], individual[:, 1], 'o')
plt.title('A random individual')
# plt.axis('equal')
plt.axis('scaled')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.savefig('figures/prob2-random-individual.pdf')
plt.show()

In [ ]:
def pairwise(iterable):
    """Iterate over the given iterable in pairs.

    pairwise([1, 2, 3, 4]) -> (1, 2), (2, 3), (3, 4)
    """
    a, b = itertools.tee(iterable)
    # Advance b one step
    next(b, None)
    return zip(a, b)

In [ ]:
def fitness(individual):
    """Evaluate the fitness of the given individual.

    Compute the Euclidean distance between every pair of cities in the individual
    and add them together.

    :param individual: An array of cities, where each city is an (x, y) pair.
    :type individual: np.ndarray with shape (n, 2)
    :return: The fitness of the individual.
    """
    return sum(np.linalg.norm(c1 - c2) for c1, c2 in pairwise(individual))